In [1]:
try:
    h2o.remove_all()
    h2o.cluster().shutdown()
except:
    pass

In [2]:
import nltk
from nltk import word_tokenize
import h2o
from h2o.estimators.word2vec import H2OWord2vecEstimator
from nltk.corpus import stopwords
from nltk import word_tokenize
from h2o.automl import H2OAutoML
import numpy as np
import pandas as pd
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.naive_bayes import H2ONaiveBayesEstimator
#from sklearn.kernel_approximation import RBFSampler
import time

In [3]:
#helper functions
#get stop words
#nltk.download()
STOP_WORDS=list(stopwords.words('english'))

def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

    


In [4]:
h2o.init(min_mem_size='29G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from C:\Users\Lanier\AppData\Roaming\Python\Python36\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Lanier\AppData\Local\Temp\tmpxlzbfg94
  JVM stdout: C:\Users\Lanier\AppData\Local\Temp\tmpxlzbfg94\h2o_Lanier_started_from_python.out
  JVM stderr: C:\Users\Lanier\AppData\Local\Temp\tmpxlzbfg94\h2o_Lanier_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_Lanier_2yp025
H2O cluster total nodes:,1
H2O cluster free memory:,27.79 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"accepting new members, healthy"


#### Load data

In [ ]:

#load data

train_data=h2o.import_file('train.csv')
#test_data=h2o.import_file('test.csv',header=1) #kagle provided this but we won't use it unless we want to submit


#### Vectorize Text (Sibi already did this, this can be skipped)

In [ ]:
#load pretrained embeddings
gl=h2o.import_file('glove.6B.300d.txt')

In [ ]:

#This converts sentenses into vectors

words_train=tokenize(train_data['question_text'])
words_test=tokenize(train_data['question_text'])
w2v_model = H2OWord2vecEstimator(pre_trained=gl)
w2v_model.train(training_frame=words_train)
train_questions= w2v_model.transform(words_train, aggregate_method = "AVERAGE")
#test_questions= w2v_model.transform(words_test, aggregate_method = "AVERAGE") #kaggle gave this file for its competition we don't need it
train_questions=train_questions.cbind(train_data['target'])
del train_data #free up ram


In [ ]:
h2o.export_file(train_questions,"vectorized_train.csv",force=True)
#h2o.export_file(test_questions,"vectorized_test.csv",force=True) #kaggle gave this file for its competition we don't need it

In [ ]:
h2o.remove_all()


#### Train models

In [5]:
train_questions=h2o.import_file("train_glove480.csv") #use pretrained by Sibi instead
#train_questions=h2o.import_file("vectorized_train.csv") #use pretrained by Sibi instead
#test_questions=h2o.import_file("vectorized_test.csv") #kaggle gave this file for its competition we don't need it
train_questions


Parse progress: |█████████████████████████████████████████████████████████| 100%


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200,C201,C202,C203,C204,C205,C206,C207,C208,C209,C210,C211,C212,C213,C214,C215,C216,C217,C218,C219,C220,C221,C222,C223,C224,C225,C226,C227,C228,C229,C230,C231,C232,C233,C234,C235,C236,C237,C238,C239,C240,C241,C242,C243,C244,C245,C246,C247,C248,C249,C250,C251,C252,C253,C254,C255,C256,C257,C258,C259,C260,C261,C262,C263,C264,C265,C266,C267,C268,C269,C270,C271,C272,C273,C274,C275,C276,C277,C278,C279,C280,C281,C282,C283,C284,C285,C286,C287,C288,C289,C290,C291,C292,C293,C294,C295,C296,C297,C298,C299,C300,C301,C302
00002165364db923c7e6,0.0510106,0.00532269,0.00755154,-0.0295723,0.245782,-0.0820922,-0.0341224,0.0510615,-0.0168528,2.36619,-0.264331,-0.0935255,0.0593031,-0.0726281,0.0512527,-0.012673,-0.00064738,0.670599,-0.108071,-0.152699,0.133436,-0.167754,0.0637519,0.00876994,-0.0673889,0.114964,-0.110361,0.148749,-0.12667,0.083761,0.0287516,0.15943,0.0502208,0.0773172,0.189724,-0.0678288,-0.0469008,0.0153311,-0.193426,-0.00298523,0.00544435,0.178721,0.0755265,-0.117596,0.13214,-0.0909577,-0.0442642,0.0459688,0.0472374,0.0183408,-0.210162,0.205587,-0.0262212,-0.0387465,0.159077,0.0748414,0.0663811,-0.0861276,0.129072,-0.0524235,-0.166051,-0.0227882,-0.0936252,0.188334,0.0332548,0.145394,-0.0623804,0.00390968,0.120383,0.0301137,-0.0244256,0.159174,0.042682,-0.0779583,0.162933,0.00124177,0.00349442,-0.0498627,0.0383422,0.164524,-0.0431332,0.176004,-0.00732792,-0.0421539,0.0640984,-0.144935,-0.312476,-0.26345,0.0135457,0.120775,-0.275555,0.15162,0.0195971,0.0873649,0.19298,-0.006572,0.0765897,0.059612,-0.0100935,-0.0550236,-0.131147,0.121238,0.0829547,-0.0487254,0.15724,-1.03149,0.0669909,-0.0637764,0.0266144,0.0291574,-0.0186974,-0.0372635,0.0119358,-0.0974339,-0.0559585,-0.0308403,0.110768,0.0789191,-0.154074,0.00651798,0.0214982,-0.0360729,-0.15425,0.0840733,0.0442981,0.0911856,0.119896,0.0221807,0.0891914,-0.0266729,-0.0838842,0.0100425,-0.0358176,-0.00042508,0.160823,-0.0226856,-0.051957,0.0159455,0.0367605,-0.069799,-0.994596,0.01978,0.2334,-0.0553575,0.0377172,-0.0619433,0.00513015,0.144915,0.0256216,-0.0176644,-0.00756514,0.2013,0.0227414,-0.0489881,-0.0606911,0.0251843,-0.0752936,-0.0310748,-0.114389,0.0866806,-0.0450352,0.160894,-0.048542,0.0495197,-0.0680022,0.0814201,0.112634,-0.160573,0.189174,-0.0610462,0.0200019,0.0288005,-0.0682022,0.0332253,-0.0331904,-0.0692218,-0.00248346,0.0413618,0.00099592,-0.0944012,0.049238,0.110165,-0.0970975,-0.239509,0.0370639,0.0349803,-0.0248835,-0.0586104,-0.00660677,-0.0899496,-0.0275635,-0.00743977,0.0612449,-0.00754838,0.0139347,0.0280194,0.0726349,-0.0485727,-0.00725315,0.119667,-0.0117509,0.200289,-0.0666595,0.0397169,0.255362,-0.0474539,0.0220977,-0.17343,0.0798993,0.0656131,-0.00518846,0.0470785,-0.0321935,-0.0666891,-0.0790958,0.140139,-0.104769,0.125647,-0.0217403,-0.0548543,0.0605896,0.0143649,0.0714595,-0.0256251,-0.0397749,-0.0650754,0.0721566,0.182636,0.0098881,0.164632,-0.110615,0.158964,0.021538,-0.0701125,-0.0777323,-0.08546,-0.002659,-0.0360412,0.089756,0.0499071,-0.0185205,0.159132,0.151317,-0.0153471,0.168736,-0.131951,0.0147602,-0.112879,-0.208412,-0.184408,0.0867634,0.0692656,0.0286499,0.00183753,0

In [6]:
#rename target column
train_questions['target']=train_questions['C302']
train_questions=train_questions.drop(['C302'])
train_questions=train_questions.drop(['C1'])
train_questions

C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,C101,C102,C103,C104,C105,C106,C107,C108,C109,C110,C111,C112,C113,C114,C115,C116,C117,C118,C119,C120,C121,C122,C123,C124,C125,C126,C127,C128,C129,C130,C131,C132,C133,C134,C135,C136,C137,C138,C139,C140,C141,C142,C143,C144,C145,C146,C147,C148,C149,C150,C151,C152,C153,C154,C155,C156,C157,C158,C159,C160,C161,C162,C163,C164,C165,C166,C167,C168,C169,C170,C171,C172,C173,C174,C175,C176,C177,C178,C179,C180,C181,C182,C183,C184,C185,C186,C187,C188,C189,C190,C191,C192,C193,C194,C195,C196,C197,C198,C199,C200,C201,C202,C203,C204,C205,C206,C207,C208,C209,C210,C211,C212,C213,C214,C215,C216,C217,C218,C219,C220,C221,C222,C223,C224,C225,C226,C227,C228,C229,C230,C231,C232,C233,C234,C235,C236,C237,C238,C239,C240,C241,C242,C243,C244,C245,C246,C247,C248,C249,C250,C251,C252,C253,C254,C255,C256,C257,C258,C259,C260,C261,C262,C263,C264,C265,C266,C267,C268,C269,C270,C271,C272,C273,C274,C275,C276,C277,C278,C279,C280,C281,C282,C283,C284,C285,C286,C287,C288,C289,C290,C291,C292,C293,C294,C295,C296,C297,C298,C299,C300,C301,target
0.0510106,0.00532269,0.00755154,-0.0295723,0.245782,-0.0820922,-0.0341224,0.0510615,-0.0168528,2.36619,-0.264331,-0.0935255,0.0593031,-0.0726281,0.0512527,-0.012673,-0.00064738,0.670599,-0.108071,-0.152699,0.133436,-0.167754,0.0637519,0.00876994,-0.0673889,0.114964,-0.110361,0.148749,-0.12667,0.083761,0.0287516,0.15943,0.0502208,0.0773172,0.189724,-0.0678288,-0.0469008,0.0153311,-0.193426,-0.00298523,0.00544435,0.178721,0.0755265,-0.117596,0.13214,-0.0909577,-0.0442642,0.0459688,0.0472374,0.0183408,-0.210162,0.205587,-0.0262212,-0.0387465,0.159077,0.0748414,0.0663811,-0.0861276,0.129072,-0.0524235,-0.166051,-0.0227882,-0.0936252,0.188334,0.0332548,0.145394,-0.0623804,0.00390968,0.120383,0.0301137,-0.0244256,0.159174,0.042682,-0.0779583,0.162933,0.00124177,0.00349442,-0.0498627,0.0383422,0.164524,-0.0431332,0.176004,-0.00732792,-0.0421539,0.0640984,-0.144935,-0.312476,-0.26345,0.0135457,0.120775,-0.275555,0.15162,0.0195971,0.0873649,0.19298,-0.006572,0.0765897,0.059612,-0.0100935,-0.0550236,-0.131147,0.121238,0.0829547,-0.0487254,0.15724,-1.03149,0.0669909,-0.0637764,0.0266144,0.0291574,-0.0186974,-0.0372635,0.0119358,-0.0974339,-0.0559585,-0.0308403,0.110768,0.0789191,-0.154074,0.00651798,0.0214982,-0.0360729,-0.15425,0.0840733,0.0442981,0.0911856,0.119896,0.0221807,0.0891914,-0.0266729,-0.0838842,0.0100425,-0.0358176,-0.00042508,0.160823,-0.0226856,-0.051957,0.0159455,0.0367605,-0.069799,-0.994596,0.01978,0.2334,-0.0553575,0.0377172,-0.0619433,0.00513015,0.144915,0.0256216,-0.0176644,-0.00756514,0.2013,0.0227414,-0.0489881,-0.0606911,0.0251843,-0.0752936,-0.0310748,-0.114389,0.0866806,-0.0450352,0.160894,-0.048542,0.0495197,-0.0680022,0.0814201,0.112634,-0.160573,0.189174,-0.0610462,0.0200019,0.0288005,-0.0682022,0.0332253,-0.0331904,-0.0692218,-0.00248346,0.0413618,0.00099592,-0.0944012,0.049238,0.110165,-0.0970975,-0.239509,0.0370639,0.0349803,-0.0248835,-0.0586104,-0.00660677,-0.0899496,-0.0275635,-0.00743977,0.0612449,-0.00754838,0.0139347,0.0280194,0.0726349,-0.0485727,-0.00725315,0.119667,-0.0117509,0.200289,-0.0666595,0.0397169,0.255362,-0.0474539,0.0220977,-0.17343,0.0798993,0.0656131,-0.00518846,0.0470785,-0.0321935,-0.0666891,-0.0790958,0.140139,-0.104769,0.125647,-0.0217403,-0.0548543,0.0605896,0.0143649,0.0714595,-0.0256251,-0.0397749,-0.0650754,0.0721566,0.182636,0.0098881,0.164632,-0.110615,0.158964,0.021538,-0.0701125,-0.0777323,-0.08546,-0.002659,-0.0360412,0.089756,0.0499071,-0.0185205,0.159132,0.151317,-0.0153471,0.168736,-0.131951,0.0147602,-0.112879,-0.208412,-0.184408,0.0867634,0.0692656,0.0286499,0.00183753,0.0677578,0.239901,-0.0

In [7]:
#convert target to category

train_questions['target']=train_questions['target'].ascharacter().asfactor()

In [8]:
len(train_questions)

1306118

In [9]:
train_questions, test=train_questions.split_frame(ratios=[.99])
sample_percent_increment=.00452513966480446927374301676
print(sample_percent_increment*len(train_questions))

5850.75217877095


In [10]:
%%capture output

x=train_questions.columns
y='target'
x.remove(y)





test_results_glm=[]
test_results_gbm=[]
test_results_DL=[]
test_results_B=[]
glm_model = H2OGeneralizedLinearEstimator(nfolds=5,family= "binomial", lambda_ = 0, compute_p_values = True,remove_collinear_columns =True)
gbm_model = H2OGradientBoostingEstimator(nfolds=5,validation_frame=test)
DL_model = H2ODeepLearningEstimator(nfolds=5,validation_frame=test)
B_model = H2ONaiveBayesEstimator(nfolds=5,validation_frame=test)
running_increment=sample_percent_increment
for i in range(30) :   
    train_questions_sam=train_questions.split_frame(ratios=[running_increment])[0]
    glm_model.train(y = y, x = x, training_frame = train_questions_sam)
    gbm_model.train(y = y, x = x, training_frame = train_questions_sam)
    DL_model.train(y = y, x = x, training_frame = train_questions_sam)
    B_model.train(y = y, x = x, training_frame = train_questions_sam)
    pred_test_glm=glm_model.predict(test)
    pred_test_gbm=gbm_model.predict(test)
    pred_test_DL=DL_model.predict(test)
    pred_test_B=B_model.predict(test)
    print(glm_model)
    print(gbm_model)
    print(DL_model)
    print(B_model)
    print("GLM acc:" + str(np.mean((pred_test_glm['predict']==test['target']).as_data_frame())))
    print("GBM acc:" + str(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame())))
    print("DL acc:" + str(np.mean((pred_test_DL['predict']==test['target']).as_data_frame())))
    print("B acc:" + str(np.mean((pred_test_B['predict']==test['target']).as_data_frame())))
    test_results_glm.append(np.mean((pred_test_glm['predict']==test['target']).as_data_frame()))
    test_results_gbm.append(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame()))
    test_results_DL.append(np.mean((pred_test_DL['predict']==test['target']).as_data_frame()))
    test_results_B.append(np.mean((pred_test_DL['predict']==test['target']).as_data_frame()))
    running_increment=running_increment+sample_percent_increment
    

In [11]:
cell_out=str(output)
text_file=open("results1.txt","w")
text_file.write(cell_out)
text_file.close()

In [12]:
print(test_results_glm)
print(test_results_gbm)
print(test_results_DL)
print(test_results_B)

[predict    0.924928
dtype: float64, predict    0.936466
dtype: float64, predict    0.927661
dtype: float64, predict    0.931456
dtype: float64, predict    0.927053
dtype: float64, predict    0.932443
dtype: float64, predict    0.93138
dtype: float64, predict    0.939502
dtype: float64, predict    0.935707
dtype: float64, predict    0.932595
dtype: float64, predict    0.933202
dtype: float64, predict    0.933354
dtype: float64, predict    0.936997
dtype: float64, predict    0.932367
dtype: float64, predict    0.935859
dtype: float64, predict    0.933581
dtype: float64, predict    0.931911
dtype: float64, predict    0.926522
dtype: float64, predict    0.930545
dtype: float64, predict    0.933657
dtype: float64, predict    0.932746
dtype: float64, predict    0.933278
dtype: float64, predict    0.930469
dtype: float64, predict    0.931228
dtype: float64, predict    0.934568
dtype: float64, predict    0.935099
dtype: float64, predict    0.935251
dtype: float64, predict    0.934568
dtype: f

In [13]:
try:
    h2o.remove_all()
    h2o.cluster().shutdown()
except:
    pass
h2o.init(min_mem_size='29G')

H2O session _sid_b5a8 closed.
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from C:\Users\Lanier\AppData\Roaming\Python\Python36\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Lanier\AppData\Local\Temp\tmpt6am3sf0
  JVM stdout: C:\Users\Lanier\AppData\Local\Temp\tmpt6am3sf0\h2o_Lanier_started_from_python.out
  JVM stderr: C:\Users\Lanier\AppData\Local\Temp\tmpt6am3sf0\h2o_Lanier_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_Lanier_fp72wb
H2O cluster total nodes:,1
H2O cluster free memory:,27.79 Gb
H2O cluster total cores:,0
H2O cluster allowed cores:,0
H2O cluster status:,"accepting new members, healthy"


In [14]:
time.sleep(120)

In [15]:
train_questions=h2o.import_file("train_paragram.csv") 

#rename target column
train_questions['target']=train_questions['C302']
train_questions=train_questions.drop(['C302'])
train_questions=train_questions.drop(['C1'])
#convert target to category
train_questions['target']=train_questions['target'].ascharacter().asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [16]:
train_questions, test=train_questions.split_frame(ratios=[.99])
sample_percent_increment=.00436452513966480446927374301676
print(sample_percent_increment*len(train_questions))

5643.313547486033


In [17]:
%%capture output

x=train_questions.columns
y='target'
x.remove(y)





test_results_glm=[]
test_results_gbm=[]
test_results_DL=[]
test_results_B=[]
glm_model = H2OGeneralizedLinearEstimator(nfolds=5,family= "binomial", lambda_ = 0, compute_p_values = True,remove_collinear_columns =True)
gbm_model = H2OGradientBoostingEstimator(nfolds=5,validation_frame=test)
DL_model = H2ODeepLearningEstimator(nfolds=5,validation_frame=test)
B_model = H2ONaiveBayesEstimator(nfolds=5,validation_frame=test)
running_increment=sample_percent_increment
for i in range(30) :   
    train_questions_sam=train_questions.split_frame(ratios=[running_increment])[0]
    glm_model.train(y = y, x = x, training_frame = train_questions_sam)
    gbm_model.train(y = y, x = x, training_frame = train_questions_sam)
    DL_model.train(y = y, x = x, training_frame = train_questions_sam)
    B_model.train(y = y, x = x, training_frame = train_questions_sam)
    pred_test_glm=glm_model.predict(test)
    pred_test_gbm=gbm_model.predict(test)
    pred_test_DL=DL_model.predict(test)
    pred_test_B=B_model.predict(test)
    print(glm_model)
    print(gbm_model)
    print(DL_model)
    print(B_model)
    print("GLM acc:" + str(np.mean((pred_test_glm['predict']==test['target']).as_data_frame())))
    print("GBM acc:" + str(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame())))
    print("DL acc:" + str(np.mean((pred_test_DL['predict']==test['target']).as_data_frame())))
    print("B acc:" + str(np.mean((pred_test_B['predict']==test['target']).as_data_frame())))
    test_results_glm.append(np.mean((pred_test_glm['predict']==test['target']).as_data_frame()))
    test_results_gbm.append(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame()))
    test_results_DL.append(np.mean((pred_test_DL['predict']==test['target']).as_data_frame()))
    test_results_B.append(np.mean((pred_test_DL['predict']==test['target']).as_data_frame()))
    running_increment=running_increment+sample_percent_increment
    

In [18]:
cell_out=str(output)
text_file=open("results2.txt","w")
text_file.write(cell_out)
text_file.close()

In [19]:
print(test_results_glm)
print(test_results_gbm)
print(test_results_DL)
print(test_results_B)

[predict    0.919982
dtype: float64, predict    0.932099
dtype: float64, predict    0.921201
dtype: float64, predict    0.92806
dtype: float64, predict    0.930117
dtype: float64, predict    0.930803
dtype: float64, predict    0.92966
dtype: float64, predict    0.925545
dtype: float64, predict    0.932404
dtype: float64, predict    0.926307
dtype: float64, predict    0.927602
dtype: float64, predict    0.930575
dtype: float64, predict    0.932175
dtype: float64, predict    0.933013
dtype: float64, predict    0.930727
dtype: float64, predict    0.928974
dtype: float64, predict    0.931642
dtype: float64, predict    0.928669
dtype: float64, predict    0.928365
dtype: float64, predict    0.927831
dtype: float64, predict    0.926383
dtype: float64, predict    0.927602
dtype: float64, predict    0.932175
dtype: float64, predict    0.927907
dtype: float64, predict    0.931108
dtype: float64, predict    0.927984
dtype: float64, predict    0.929965
dtype: float64, predict    0.931032
dtype: fl

In [20]:
try:
    h2o.remove_all()
    h2o.cluster().shutdown()
except:
    pass
h2o.init(min_mem_size='29G')

H2O session _sid_af20 closed.
Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from C:\Users\Lanier\AppData\Roaming\Python\Python36\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Lanier\AppData\Local\Temp\tmpucexl9qr
  JVM stdout: C:\Users\Lanier\AppData\Local\Temp\tmpucexl9qr\h2o_Lanier_started_from_python.out
  JVM stderr: C:\Users\Lanier\AppData\Local\Temp\tmpucexl9qr\h2o_Lanier_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Chicago
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_Lanier_ec40v3
H2O cluster total nodes:,1
H2O cluster free memory:,27.79 Gb
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"accepting new members, healthy"


In [21]:
time.sleep(120)

In [22]:
train_questions=h2o.import_file("train_google_news.csv") 

#rename target column
train_questions['target']=train_questions['C302']
train_questions=train_questions.drop(['C302'])
train_questions=train_questions.drop(['C1'])
#convert target to category
train_questions['target']=train_questions['target'].ascharacter().asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [23]:
train_questions, test=train_questions.split_frame(ratios=[.99])
sample_percent_increment=.00436452513966480446927374301676
print(sample_percent_increment*len(train_questions))

5644.086068435754


In [24]:
%%capture output

x=train_questions.columns
y='target'
x.remove(y)





test_results_glm=[]
test_results_gbm=[]
test_results_DL=[]
test_results_B=[]
glm_model = H2OGeneralizedLinearEstimator(nfolds=5,family= "binomial", lambda_ = 0, compute_p_values = True,remove_collinear_columns =True)
gbm_model = H2OGradientBoostingEstimator(nfolds=5,validation_frame=test)
DL_model = H2ODeepLearningEstimator(nfolds=5,validation_frame=test)
B_model = H2ONaiveBayesEstimator(nfolds=5,validation_frame=test)
running_increment=sample_percent_increment
for i in range(30) :   
    train_questions_sam=train_questions.split_frame(ratios=[running_increment])[0]
    glm_model.train(y = y, x = x, training_frame = train_questions_sam)
    gbm_model.train(y = y, x = x, training_frame = train_questions_sam)
    DL_model.train(y = y, x = x, training_frame = train_questions_sam)
    B_model.train(y = y, x = x, training_frame = train_questions_sam)
    pred_test_glm=glm_model.predict(test)
    pred_test_gbm=gbm_model.predict(test)
    pred_test_DL=DL_model.predict(test)
    pred_test_B=B_model.predict(test)
    print(glm_model)
    print(gbm_model)
    print(DL_model)
    print(B_model)
    print("GLM acc:" + str(np.mean((pred_test_glm['predict']==test['target']).as_data_frame())))
    print("GBM acc:" + str(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame())))
    print("DL acc:" + str(np.mean((pred_test_DL['predict']==test['target']).as_data_frame())))
    print("B acc:" + str(np.mean((pred_test_B['predict']==test['target']).as_data_frame())))
    test_results_glm.append(np.mean((pred_test_glm['predict']==test['target']).as_data_frame()))
    test_results_gbm.append(np.mean((pred_test_gbm['predict']==test['target']).as_data_frame()))
    test_results_DL.append(np.mean((pred_test_DL['predict']==test['target']).as_data_frame()))
    test_results_B.append(np.mean((pred_test_DL['predict']==test['target']).as_data_frame()))
    running_increment=running_increment+sample_percent_increment
    

In [25]:
print(test_results_glm)
print(test_results_gbm)
print(test_results_DL)
print(test_results_B)

[predict    0.917806
dtype: float64, predict    0.92584
dtype: float64, predict    0.925299
dtype: float64, predict    0.926149
dtype: float64, predict    0.932252
dtype: float64, predict    0.926535
dtype: float64, predict    0.929471
dtype: float64, predict    0.928698
dtype: float64, predict    0.929471
dtype: float64, predict    0.932484
dtype: float64, predict    0.927849
dtype: float64, predict    0.935419
dtype: float64, predict    0.931788
dtype: float64, predict    0.928467
dtype: float64, predict    0.929857
dtype: float64, predict    0.929934
dtype: float64, predict    0.930861
dtype: float64, predict    0.928544
dtype: float64, predict    0.93202
dtype: float64, predict    0.928158
dtype: float64, predict    0.931093
dtype: float64, predict    0.92893
dtype: float64, predict    0.929625
dtype: float64, predict    0.93063
dtype: float64, predict    0.930475
dtype: float64, predict    0.928698
dtype: float64, predict    0.930784
dtype: float64, predict    0.933102
dtype: floa

In [26]:
cell_out=str(output)
text_file=open("results3.txt","w")
text_file.write(cell_out)
text_file.close()

In [27]:
try:
    h2o.remove_all()
    h2o.cluster().shutdown()
except:
    pass

H2O session _sid_a880 closed.


#### Kernal techniques

In [ ]:

#look at using RBF kernal
train_questions=pd.read_csv("train_google_news.csv", header=-1)


In [ ]:
try:
    train_questions=train_questions.drop([301], axis=1)
except:
    pass    
try:
    train_questions=train_questions.drop([0], axis=1)
except:
    pass

In [ ]:
train_questions

In [ ]:
train_questions=train_questions.astype('float32')
train_questions=train_questions.dropna()

In [ ]:
rbf_feature=RBFSampler(gamma=1, random_state=1)
X_features=rbf_feature.fit_transform(train_questions)

In [ ]:
X_features=pd.DataFrame(X_features)
X_features

In [ ]:
try:
    del train_questions
    del rbf_feature
except:
    pass
Y= pd.read_csv("train_google_news.csv", header=-1,usecols = [301])

In [ ]:

train_questions=pd.concat([X_features,Y],axis=1)

In [ ]:
train_questions

In [ ]:
time.sleep(5)